In [1]:
import torch
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [2]:
import pandas as pd
train_df = pd.read_csv('train.csv')

In [37]:
### 데이터 증강
import itertools
import random
from tqdm import tqdm

def hamming_distance(a, b):
    return sum(x != y for x, y in zip(a, b))

def augment_train_df(train_df, max_aug=2):
    """train_df에서 각 row별로 hard negative 증강(해밍거리 1~2), ID 변경"""
    augmented_rows = []
    for _, row in tqdm(train_df.iterrows(), total=len(train_df)):
        orig_perm = tuple(row[f'answer_{i}'] for i in range(4))
        all_perms = list(itertools.permutations(range(4)))
        hard_perms = [p for p in all_perms if 1 <= hamming_distance(orig_perm, p) <= 2]
        if not hard_perms:
            continue
        selected = random.sample(hard_perms, min(max_aug, len(hard_perms)))
        sentences = [row[f'sentence_{i}'] for i in range(4)]
        for idx, perm in enumerate(selected, 1):
            aug_sentences = [sentences[i] for i in perm]
            aug_row = {f'sentence_{i}': s for i, s in enumerate(aug_sentences)}
            for i in range(4):
                aug_row[f'answer_{i}'] = perm[i]
            # 증강 ID: 원본ID_1, _2, ...
            aug_row['ID'] = f"{row['ID']}_{idx}"
            augmented_rows.append(aug_row)
    aug_df = pd.DataFrame(augmented_rows)
    # 증강 데이터와 train_df 합치기 (원본은 ID 그대로, 증강은 _N)
    train_df_augmented = pd.concat([train_df, aug_df], ignore_index=True)
    return train_df_augmented




In [38]:
train_aug_df = augment_train_df(train_df, max_aug=2)

100%|██████████| 7351/7351 [00:00<00:00, 9732.78it/s] 


In [40]:
perm_counts = count_all_permutations(train_aug_df)
for k, v in perm_counts.items():
    print(k, ":", v)

(0, 1, 2, 3) : 942
(0, 1, 3, 2) : 908
(0, 2, 1, 3) : 951
(0, 2, 3, 1) : 893
(0, 3, 1, 2) : 909
(0, 3, 2, 1) : 888
(1, 0, 2, 3) : 922
(1, 0, 3, 2) : 912
(1, 2, 0, 3) : 942
(1, 2, 3, 0) : 922
(1, 3, 0, 2) : 888
(1, 3, 2, 0) : 913
(2, 0, 1, 3) : 940
(2, 0, 3, 1) : 931
(2, 1, 0, 3) : 914
(2, 1, 3, 0) : 923
(2, 3, 0, 1) : 921
(2, 3, 1, 0) : 908
(3, 0, 1, 2) : 920
(3, 0, 2, 1) : 893
(3, 1, 0, 2) : 948
(3, 1, 2, 0) : 900
(3, 2, 0, 1) : 938
(3, 2, 1, 0) : 927


In [ ]:
####



In [10]:
!rm -rf ~/.cache/huggingface

df = train_df.copy().head(3)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [29]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, set_seed
from tqdm import tqdm
import torch
import random
import numpy as np

SEED = 42  # 원하는 정수

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
set_seed(SEED)  # transformers 내부 시드

MODEL_NAME = "MLP-KTLim/llama-3-Korean-Bllossom-8B"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype="auto",
    device_map="auto"
)



generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=300,
    temperature=1.1,
    do_sample=True,
    top_p=0.92,
    repetition_penalty=1.15
)

def paraphrase_one(sentence):
    prompt = (
        "llm 학습용 데이터를 증강할거야. 아래 문장을 의미를 바꾸지 않고 다양한 표현으로 paraphrasing 해줘.\n"
        "예시:\n"
        "문장: 인공지능은 미래 사회를 변화시킬 중요한 기술이다.\n"
        "출력: 미래 사회를 바꿀 핵심 기술 중 하나가 바로 인공지능이다.\n"
        "문장: 데이터 분석은 기업 의사결정에 큰 영향을 준다.\n"
        "출력: 기업이 의사결정을 내릴 때 데이터 분석이 중요한 역할을 한다.\n"
        f"문장: {sentence}\n"
        "출력:"
    )

    output = generator(prompt, num_return_sequences=1)
    text = output[0]['generated_text'].replace(prompt, '').strip()
    # 출력 뒤 첫 줄만 추출
    clean = text.split('\n')[0].lstrip('-').strip()
    return clean


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


In [38]:
train_df

,ID,sentence_0,sentence_1,sentence_2,sentence_3,answer_0,answer_1,answer_2,answer_3
0,TRAIN_0000,블록체인 기술은 투표 과정의 투명성을 크게 향상시킬 수 있다.,"이러한 특성은 유권자들에게 신뢰를 제공하며, 민주적 참여를 촉진하는 데 기여할 수 있다.",결과적으로 블록체인 기반의 투표 시스템은 공정하고 신뢰할 수 있는 선거 환경을 조성...,각 투표는 변경 불가능한 기록으로 저장되어 조작의 가능성을 원천적으로 차단한다.,0,3,1,2
1,TRAIN_0001,줄거리 자동 생성의 인공지능 알고리즘은 대량의 텍스트 데이터를 분석하여 핵심 정보를...,"결과적으로, 이러한 기술은 사용자에게 신속하고 효율적인 정보 전달을 가능하게 한다.",생성된 줄거리는 원본 텍스트의 의미를 유지하면서도 간결하게 요약된 형태로 제공된다.,"이 알고리즘은 자연어 처리 기술을 활용하여 문맥을 이해하고, 주요 사건과 등장인물을...",0,3,2,1
2,TRAIN_0002,"마지막으로, 키친타올을 보관할 때는 쉽게 접근할 수 있는 곳에 두어 낭비를 방지하는...",재사용 가능한 천이나 스펀지를 활용하면 키친타올의 필요성을 줄일 수 있다.,물기를 제거할 때는 가볍게 눌러주어 과도한 사용을 피할 수 있다.,키친타올을 절약하는 첫걸음은 필요한 양만큼만 사용하는 것이다.,3,2,1,0
3,TRAIN_0003,책의 페이지가 손상되지 않도록 수직으로 세워 두거나 평평하게 눕혀 보관하는 것이 좋다.,"정기적으로 먼지를 털어내고, 곰팡이나 해충의 발생 여부를 점검하는 것이 중요하다.",종이책은 직사광선이 닿지 않는 서늘하고 건조한 장소에 보관해야 한다.,"필요할 경우, 책을 보호하기 위해 커버를 씌우거나 전용 보관함에 넣는 방법도 고려할...",2,0,1,3
4,TRAIN_0004,"인공지능 모델은 반복적인 실험을 통해 지속적으로 학습하며, 이를 통해 발견의 정확성...",인공지능은 대량의 데이터를 분석하여 숨겨진 패턴과 상관관계를 발견하는 데 강력한 도...,"결국, 인공지능의 지원은 과학적 발견의 속도와 효율성을 혁신적으로 변화시킬 수 있는...",이러한 분석 결과는 연구자들에게 새로운 가설을 제시하고 실험 설계를 개선하는 데 기...,1,3,0,2
...,...,...,...,...,...,...,...,...,...
7346,TRAIN_7346,인식론적 겸손은 자신의 지식의 한계를 인정하는 태도를 의미한다.,이러한 겸손은 새로운 정보와 관점을 수용하는 데 도움을 준다.,"지적 확신은 특정한 믿음이나 주장을 강하게 지지하는 태도로, 자신감 있는 결정을 가...","두 가지 태도의 균형은 비판적 사고를 촉진하고, 더 깊은 이해를 추구하는 데 필수적이다.",0,1,2,3
7347,TRAIN_7347,거래 후에는 반드시 영수증이나 거래 내역을 기록하여 추후 문제 발생 시 증거로 활용...,중고 거래를 시작하기 전에 판매자와 구매자의 신뢰성을 확인하는 것이 중요하다.,거래 플랫폼의 리뷰와 평점을 참고하여 안전한 거래 상대를 선택할 수 있다.,"상품의 상태와 가격을 명확히 확인한 후, 직접 만나서 거래하는 것이 바람직하다.",1,2,3,0
7348,TRAIN_7348,"결국, 소수자의 권리가 존중될 때 다수의 의견도 더욱 풍부해질 수 있다.","이 두 가지 원칙은 때때로 충돌할 수 있으며, 균형을 찾는 것이 필요하다.",소수자의 권리 보호는 민주주의의 핵심 요소로 여겨진다.,다수결 원칙은 사회의 다양한 의견을 반영하는 중요한 메커니즘이다.,2,3,1,0
7349,TRAIN_7349,인간의 유전자를 수정하는 것이 자연의 섭리를 거스르는 행위로 여겨지기도 한다.,"결과적으로, 이러한 논의는 과학적 진보와 도덕적 책임 사이의 균형을 찾는 데 중요한...",유전자 편집 기술은 생명과학 분야에서 혁신적인 발전을 가져왔다.,그러나 이 기술의 윤리적 측면에 대한 논란이 끊이지 않고 있다.,2,3,0,1


In [ ]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, set_seed
from tqdm import tqdm
import torch
import random
import numpy as np

# --- 시드 고정 ---
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
set_seed(SEED)

MODEL_NAME = "MLP-KTLim/llama-3-Korean-Bllossom-8B"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype="auto",
    device_map="auto"
)

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=300,
    temperature=1.1,
    do_sample=True,
    top_p=0.92,
    repetition_penalty=1.15
)

BATCH_SIZE = 12  # GPU VRAM 따라 4~32 조정

def make_prompt(sentence):
    return (
        "llm 학습용 데이터를 증강할거야. 아래 문장을 의미를 바꾸지 않고 다양한 표현으로 paraphrasing 해줘.\n"
        "예시:\n"
        "문장: 인공지능은 미래 사회를 변화시킬 중요한 기술이다.\n"
        "출력: 미래 사회를 바꿀 핵심 기술 중 하나가 바로 인공지능이다.\n"
        "문장: 데이터 분석은 기업 의사결정에 큰 영향을 준다.\n"
        "출력: 기업이 의사결정을 내릴 때 데이터 분석이 중요한 역할을 한다.\n"
        f"문장: {sentence}\n"
        "출력:"
    )

# === train.csv 읽기 ===
df = train_df.copy()
augmented_rows = []

# 1. paraphrase할 (행idx, 컬럼명, 원문) 리스트로 모으기
to_paraphrase = []
row_ids = []
cols = [c for c in df.columns if c.startswith("sentence_")]
for idx, row in df.iterrows():
    for col in cols:
        to_paraphrase.append((idx, col, row[col]))

# 2. 배치로 한 번에 inference
prompts = [make_prompt(x[2]) for x in to_paraphrase]
paraphrased_results = []
for i in tqdm(range(0, len(prompts), BATCH_SIZE)):
    batch_prompts = prompts[i:i+BATCH_SIZE]
    outputs = generator(batch_prompts)
    for j, out in enumerate(outputs):
        text = out["generated_text"].replace(batch_prompts[j], '').strip()
        paraphrased = text.split('\n')[0].lstrip('-').strip()
        paraphrased_results.append(paraphrased)

# 3. 증강된 행 만들기
result_idx = 0
for idx, row in df.iterrows():
    new_row = row.copy()
    new_row["ID"] = str(row["ID"]) + "_aug"
    for col in cols:
        new_row[col] = paraphrased_results[result_idx]
        result_idx += 1
    augmented_rows.append(new_row)

# 4. 원본+증강 합치기
aug_df = pd.concat([df, pd.DataFrame(augmented_rows)], ignore_index=True)
aug_df.to_csv("train_augmented_rows_batch.csv", index=False)
print("증강+배치 데이터 저장 완료! → train_augmented_rows_batch.csv")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cpu
  0%|          | 0/2451 [00:00<?, ?it/s]